# testing with ECG

In [2]:
# Train sklearn Random Forest on Iris dataset and save to Redis
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sample_converter import sklearn_sample_to_dict
# Import conversion functions
from skforest_to_forest import sklearn_forest_to_forest
from redis_helpers.forest import store_forest
from aeon.datasets import load_classification
from sklearn.utils import Bunch
import numpy as np
import math
from sklearn.model_selection import train_test_split
from cost_function import cal_sigmas

def load_aeon_dataset_padded(dataset_name, return_as_bunch=False, feature_prefix="t"):
	X_train, y_train = load_classification(dataset_name, split="train")
	X_test, y_test = load_classification(dataset_name, split="test")

	print(f"Dataset: {dataset_name}")
	print(f"Training set: {X_train.shape} samples")
	print(f"Test set: {X_test.shape} samples")
	print(f"Series length: {X_train.shape[2]} time points")
	print(f"Classes: {np.unique(np.concatenate([y_train, y_test]))}")

	if return_as_bunch:
		# Reshape data: (n_samples, n_channels, n_timepoints) -> (n_samples, n_timepoints)
		X_train_2d = X_train.reshape(X_train.shape[0], -1)
		X_test_2d = X_test.reshape(X_test.shape[0], -1)

		# Combine train and test
		X_combined = np.vstack([X_train_2d, X_test_2d])
		y_combined = np.concatenate([y_train, y_test])

		# Calculate padding width based on series length
		series_length = X_combined.shape[1]
		# Number of digits needed (e.g., for 2000 points: indices 0-1999, max is 1999 = 4 digits)
		padding_width = len(str(series_length - 1))

		# Create zero-padded feature names
		feature_names = [f"{feature_prefix}_{i:0{padding_width}d}" for i in range(series_length)]

		print(f"Feature name format: {padding_width} digits")
		print(f"Sample feature names: {feature_names[:5]}...{feature_names[-5:]}")

		bunch = Bunch(
			data=X_combined,
			target=y_combined,
			feature_names=feature_names,
			target_names=np.unique(y_combined).astype(str),
			DESCR=f"Aeon {dataset_name} dataset - univariate time series classification",
			train_indices=np.arange(len(X_train_2d)),
			test_indices=np.arange(len(X_train_2d), len(X_train_2d) + len(X_test_2d)),
			original_shape=X_train.shape[1:],
			padding_width=padding_width  # Store for reference
		)
		return bunch

	return X_train, X_test, y_train, y_test

names_dataset = ['ECG200']
dt = {}
for name_dataset in names_dataset:
	tmp = load_aeon_dataset_padded(name_dataset, True)
	dt[name_dataset] = {"X" : tmp.data, "y": tmp.target, "feature_names": tmp.feature_names, "class_names": tmp.target_names, "string_labels" : tmp.target, "X_train": tmp.data[tmp.train_indices], "X_test": tmp.data[tmp.test_indices], "y_train": tmp.target[tmp.train_indices], "y_test": tmp.target[tmp.test_indices], "train_indices": tmp.train_indices, "test_indices": tmp.test_indices}


	print(f"Training set: {dt["X_train"].shape[0]} samples")
	print(f"Test set: {dt["X_test"].shape[0]} samples")

	sigmas_all = cal_sigmas(dt["X_train"], dt["X_test"], dt["feature_names"])

	list_sample_radis = []
	for i in range(len(X_test)):
		sample= db.get_metadata(i)
		list.append((sample_dict, test_index))

	lista_ar = [ar.bitmpaToICF() for ar in DB ]
	from cost_function import cost_function
	pot_rob = {}
	for (sample_dict, index) in list_sample_radis:
		for icf in lista_ar:
			c = cost_function(sample_dict, sigmas=sigmas_all, icf=icf, sample_index=index, verbose=True)
			print(f"Sample {index} Cost: {c}")

			pot_rob[index] = {'cost': c, 'icf':icf}
		robustness = max(pot_rob[index]['cost'])
		print(f"Overall Robustness: {robustness} for sample {index}")
	pot_rob.to_csv(f'{name_dataset}_robustness.csv')


Dataset: ECG200
Training set: (100, 1, 96) samples
Test set: (100, 1, 96) samples
Series length: 96 time points
Classes: ['-1' '1']
Feature name format: 2 digits
Sample feature names: ['t_00', 't_01', 't_02', 't_03', 't_04']...['t_91', 't_92', 't_93', 't_94', 't_95']


KeyError: 'X_train'

NameError: name 'X' is not defined

In [ ]:
ecg = load_aeon_dataset_padded('ECG200', True)
# Load ecg dataset

from sklearn.model_selection import train_test_split


X, y = ecg.data, ecg.target
feature_names = ecg.feature_names
class_names = ecg.target_names
string_labels = ecg.target


print(f"ECG dataset: {X.shape} samples, {len(feature_names)} features")
print(f"Features: {feature_names}")
print(f"Classes: {class_names}")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, string_labels, test_size=0.3, random_state=42)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
from cost_function import cal_sigmas

sigmas_all = cal_sigmas(X_train, X_test, feature_names)

list_sample_radis = []
for i in range(len(X_test)):



# calc sigmas + and - on train dataset

In [ ]:
from cost_function import cal_sigmas

sigmas_all = cal_sigmas(X_train, X_test, feature_names)

# Creation of ICF

In [ ]:
import random
icf = {}
for k in feature_names:
	int_sub = random.uniform(-1, 0)
	icf[k] = (int_sub, int_sub*1.2)
icf

# Cost function

In [ ]:
from cost_function import cost_function

cost_function(sample_dict, sigmas=sigmas_all, icf=icf, sample_index=sample_index, verbose=True)

In [ ]:
len(feature_names)

# plotting for sez 3

In [ ]:
# Plot ECG200 time series, corridor, and intervals outside
import numpy as np
import matplotlib.pyplot as plt

# Use the first test sample from ECG200
sample_idx = 1
series = X_test[sample_idx]
feature_names_ecg = feature_names
n_points_ecg = len(series)
x_ecg = np.arange(n_points_ecg)

# Plot 1: ECG time series only
plt.figure(figsize=(10, 4))
plt.plot(x_ecg, series, label='ECG200 Time Series', color='blue')
plt.title('ECG200 Time Series (Test Sample)')
plt.xlabel('Time Index')
plt.ylabel('Value')
plt.legend()
plt.show()


# Plot 2: ECG time series with corridor and intervals outside
corridor_width_ecg = 0.3
upper_corridor_ecg = series + corridor_width_ecg
lower_corridor_ecg = series - corridor_width_ecg

plt.figure(figsize=(10, 4))
plt.plot(x_ecg, series, label='ECG200 Time Series', color='blue')
plt.fill_between(x_ecg, lower_corridor_ecg, upper_corridor_ecg, color='orange', alpha=0.3, label='Darwiche Reason')

# Generate random intervals outside the corridor
num_intervals_ecg = 3
np.random.seed(123)
for i in range(num_intervals_ecg):
	idx_start = np.random.randint(0, n_points_ecg // 2)
	idx_end = idx_start + np.random.randint(5, 15)
	if idx_end >= n_points_ecg:
		idx_end = n_points_ecg - 1
	if np.random.rand() > 0.5:
		y_base = upper_corridor_ecg[idx_start:idx_end] + np.random.uniform(0.2, 0.5)
	else:
		y_base = lower_corridor_ecg[idx_start:idx_end] - np.random.uniform(0.2, 0.5)
	plt.plot(x_ecg[idx_start:idx_end], y_base, color='orange', linewidth=4, alpha=0.3)

plt.title('ECG200 Time Series with Darwiche Reason')
plt.xlabel('Time Index')
plt.ylabel('Value')
plt.legend()
plt.show()

# Plot 3 our maximal reason corridor only
np.random.seed(2025)
base_width_maximal = 0.3
random_offsets_maximal = np.random.uniform(-0.1, 0.1, n_points_ecg)
upper_maximal = series + base_width_maximal + random_offsets_maximal
lower_maximal = series - base_width_maximal + random_offsets_maximal

plt.figure(figsize=(10, 4))
plt.plot(x_ecg, series, label='ECG200 Time Series', color='blue')
plt.fill_between(x_ecg, lower_corridor_ecg, upper_corridor_ecg, color='purple', alpha=0.3, label='Maximal Reason')
plt.title('ECG200 Time Series with our Maximal Reason')
plt.xlabel('Time Index')
plt.ylabel('Value')
plt.legend()
plt.show()